# This code is to gather the information of the energy consumption of the whole training process of different models.

## import the required libraries

In [54]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
from d2l import torch as d2l
import numpy as np
import pandas as pd
from ptflops import get_model_complexity_info
from pathlib import Path
import os
import time
import pynvml
import threading
import queue
from functools import wraps

## find the path

In [55]:
'''find the Model path'''
# find the current path
from pathlib import Path

# find the current path
current_path = Path.cwd()
print('The current path is:', current_path)

# find the data path
data_path = Path(current_path / 'ModelsData')
print('The data path is:', data_path)

The current path is: /root/autodl-tmp/GreenAI/3080
The data path is: /root/autodl-tmp/GreenAI/3080/ModelsData


## Models

### generate the data paths

In [56]:
models_name = ['googlenet_origin']

In [57]:
DataList = [Path(f"{data_path}/{i}") for i in models_name]
print(DataList)

[PosixPath('/root/autodl-tmp/GreenAI/3080/ModelsData/googlenet_origin')]


### create the models

#### GoogleNet Models

##### GoogleNet Model(orinigal)

In [58]:
def timeit(layer_name):
    """装饰器，用于记录方法的执行时间，并存储到 self.layer_time 中"""
    def decorator(func):
        @wraps(func)  # 使用 wraps 修饰 wrapper 函数
        def wrapper(self, *args, **kwargs):
            start_t = time.time()
            result = func(self, *args, **kwargs)
            torch.cuda.synchronize()  # 确保时间精确性
            end_t = time.time()
            self.layer_time[layer_name] = (start_t, end_t)
            return result
        return wrapper
    return decorator

class Inception(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_conv = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_conv1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_conv2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_conv1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_conv2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_mp = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_conv = nn.Conv2d(in_channels, c4, kernel_size=1)

        # initial the relu layer
        self.p1_relu = nn.ReLU()
        self.p2_relu1 = nn.ReLU()
        self.p2_relu2 = nn.ReLU()
        self.p3_relu1 = nn.ReLU()
        self.p3_relu2 = nn.ReLU()
        self.p4_relu = nn.ReLU()

        self.layer_time = {}

    @timeit('p1_conv')
    def forward_p1_conv(self, x):
        return self.p1_conv(x)

    @timeit('p1_relu')
    def forward_p1_relu(self, x):
        return self.p1_relu(x)

    @timeit('p2_1_conv')
    def forward_p2_1_conv(self, x):
        return self.p2_conv1(x)

    @timeit('p2_1_relu')
    def forward_p2_1_relu(self, x):
        return self.p2_relu1(x)

    @timeit('p2_2_conv')
    def forward_p2_2_conv(self, x):
        return self.p2_conv2(x)

    @timeit('p2_2_relu')
    def forward_p2_2_relu(self, x):
        return self.p2_relu2(x)

    @timeit('p3_1_conv')
    def forward_p3_1_conv(self, x):
        return self.p3_conv1(x)

    @timeit('p3_1_relu')
    def forward_p3_1_relu(self, x):
        return self.p3_relu1(x)

    @timeit('p3_2_conv')
    def forward_p3_2_conv(self, x):
        return self.p3_conv2(x)

    @timeit('p3_2_relu')
    def forward_p3_2_relu(self, x):
        return self.p3_relu2(x)

    @timeit('p4_pool')
    def forward_p4_pool(self, x):
        return self.p4_mp(x)

    @timeit('p4_conv')
    def forward_p4_conv(self, x):
        return self.p4_conv(x)

    @timeit('p4_relu')
    def forward_p4_relu(self, x):
        return self.p4_relu(x)
    
    @timeit('concat')
    def forward_concat(self, p1_out, p2_out, p3_out, p4_out):
        return torch.cat((p1_out, p2_out, p3_out, p4_out), dim=1)
    
    def forward(self, x):
        p1_out = self.forward_p1_conv(x)
        p1_out = self.forward_p1_relu(p1_out)

        p2_out = self.forward_p2_1_conv(x)
        p2_out = self.forward_p2_1_relu(p2_out)
        p2_out = self.forward_p2_2_conv(p2_out)
        p2_out = self.forward_p2_2_relu(p2_out)

        p3_out = self.forward_p3_1_conv(x)
        p3_out = self.forward_p3_1_relu(p3_out)
        p3_out = self.forward_p3_2_conv(p3_out)
        p3_out = self.forward_p3_2_relu(p3_out)

        p4_out = self.forward_p4_pool(x)
        p4_out = self.forward_p4_conv(p4_out)
        p4_out = self.forward_p4_relu(p4_out)

        out = self.forward_concat(p1_out, p2_out, p3_out, p4_out)

        # p1 = F.relu(self.p1_1(x))
        # p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        # p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        # p4 = F.relu(self.p4_2(self.p4_1(x)))
        
        # 在通道维度上连结输出
        return out
    
def Googlenet(img_channel, num_labels):
    b1 = nn.Sequential(nn.Conv2d(img_channel, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
    b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, num_labels))
    return net

## Call the Models

In [59]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')
print('The device is:', device)

# check if mps on macbook is availabel
# print(torch.backends.mps.is_available())  # 检查 MPS 是否可用
# print(torch.backends.mps.is_built())      # 检查 MPS 是否已编译

The device is: cuda


In [60]:
# create list to store all the parameters and the number of MACs, be careful of the different datasets
# to avoid the error of the number of input channels and any other mistake, try to use different dictionaries to store each dataset
# create different empty dictionary
macs_f = {}
paras_f = {}
macs_c100 = {}
paras_c100 = {}
macs_c10 = {}
paras_c10 = {}

### usea function to call the models

In [61]:
# create a function for all the models to run
# image channel for fashion mnist 
channel_f = 1
# image channel for cifar100 and cifar10
channel_c = 3

# number of labels for fashion mnist
num_labels_f = 10
# number of labels for cifar100 
num_labels_c100 = 100
# number of labels for cifar10
num_labels_c10 = 10

def get_model_info(model, img_channel, num_labels):
    model_ini = model.__name__
    print(f'The model name is {model_ini}')

    net = model(img_channel, num_labels)
    
    return net

### import all models

In [62]:
# GoogleNet
# GoogleNet
googlenet_f = get_model_info(Googlenet, channel_f, num_labels_f)
googlenet_c100 = get_model_info(Googlenet, channel_c, num_labels_c100)
googlenet_c10 = get_model_info(Googlenet, channel_c, num_labels_c10)

The model name is Googlenet
The model name is Googlenet
The model name is Googlenet


## Import the Datasets for training

In [63]:
print(models_name)

['googlenet_origin']


In [64]:
# create model list according to models_name order
models_f_list = [googlenet_f]

models_c100_list = [googlenet_c100]

models_c10_list = [googlenet_c10]

### show the output size of each layers after the picture is passed through the model

In [65]:
X_f = torch.randn(size=(1, 1, 224, 224), dtype=torch.float32) # fashion mnist

# get all the layers' name in the forward step during the training process
IncepBlock = ['p1_conv', 'p1_relu', 'p2_1_conv', 'p2_1_relu', 'p2_2_conv', 'p2_2_relu', 'p3_1_conv', 'p3_1_relu', 'p3_2_conv', 'p3_2_relu', 'p4_pool', 'p4_conv', 'p4_relu']
layerlist_googlenet = []
SeqNum = 0
for block in googlenet_f:
    blockname = block.__class__.__name__
    if blockname == 'Sequential':
        # set a number for each inception block in each sequential block
        IncepNum = 0
        for subblock in block:
            subblockname = subblock.__class__.__name__
            if subblockname == 'Inception':
                for inceplayer in IncepBlock:
                    layerlist_googlenet.append(f'{blockname}_{SeqNum}_{subblockname}_{IncepNum}_{inceplayer}')
                IncepNum += 1
            else:
                layerlist_googlenet.append(f'{blockname}_{SeqNum}_{subblockname}')
    else:
        layerlist_googlenet.append(blockname)
    SeqNum += 1 

print(f'layerlist_googlenet: {layerlist_googlenet}')


layerlist_googlenet: ['Sequential_0_Conv2d', 'Sequential_0_ReLU', 'Sequential_0_MaxPool2d', 'Sequential_1_Conv2d', 'Sequential_1_ReLU', 'Sequential_1_Conv2d', 'Sequential_1_ReLU', 'Sequential_1_MaxPool2d', 'Sequential_2_Inception_0_p1_conv', 'Sequential_2_Inception_0_p1_relu', 'Sequential_2_Inception_0_p2_1_conv', 'Sequential_2_Inception_0_p2_1_relu', 'Sequential_2_Inception_0_p2_2_conv', 'Sequential_2_Inception_0_p2_2_relu', 'Sequential_2_Inception_0_p3_1_conv', 'Sequential_2_Inception_0_p3_1_relu', 'Sequential_2_Inception_0_p3_2_conv', 'Sequential_2_Inception_0_p3_2_relu', 'Sequential_2_Inception_0_p4_pool', 'Sequential_2_Inception_0_p4_conv', 'Sequential_2_Inception_0_p4_relu', 'Sequential_2_Inception_1_p1_conv', 'Sequential_2_Inception_1_p1_relu', 'Sequential_2_Inception_1_p2_1_conv', 'Sequential_2_Inception_1_p2_1_relu', 'Sequential_2_Inception_1_p2_2_conv', 'Sequential_2_Inception_1_p2_2_relu', 'Sequential_2_Inception_1_p3_1_conv', 'Sequential_2_Inception_1_p3_1_relu', 'Sequentia

### load all the datas:  
    1. FashionMNIST
    2. CIFAR100
    3. CIFAR10

In [66]:
# load the data
# fashion mnist
def get_dataloader_workers():
    """Use 4 processes to read the data.

    Defined in :numref:`sec_utils`"""
    return 4

def load_data_fashion_mnist(batch_size, resize=None):
    """下载Fashion-MNIST数据集, 然后将其加载到内存中

    Defined in :numref:`sec_fashion_mnist`"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="../data", train=False, transform=trans, download=True)
    return (torch.utils.data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=get_dataloader_workers()),
            torch.utils.data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=get_dataloader_workers()))

def load_data_cifar100(batch_size, resize=None):
    """Download the Fashion-MNIST dataset and then load it into memory.

    Defined in :numref:`sec_utils`"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    # import the cifar100 dataset
    cifar_train = torchvision.datasets.CIFAR100(
        root="../data", train=True, transform=trans, download=True)
    cifar_test = torchvision.datasets.CIFAR100(
        root="../data", train=False, transform=trans, download=True)
    return (torch.utils.data.DataLoader(cifar_train, batch_size, shuffle=True,
                                        num_workers=get_dataloader_workers()),
            torch.utils.data.DataLoader(cifar_test, batch_size, shuffle=False,
                                        num_workers=get_dataloader_workers()))
    
def load_data_cifar10(batch_size, resize=None):
    """Download the Fashion-MNIST dataset and then load it into memory.

    Defined in :numref:`sec_utils`"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    # import the cifar100 dataset
    cifar_train = torchvision.datasets.CIFAR10(
        root="../data", train=True, transform=trans, download=True)
    cifar_test = torchvision.datasets.CIFAR10(
        root="../data", train=False, transform=trans, download=True)
    return (torch.utils.data.DataLoader(cifar_train, batch_size, shuffle=True,
                                        num_workers=get_dataloader_workers()),
            torch.utils.data.DataLoader(cifar_test, batch_size, shuffle=False,
                                        num_workers=get_dataloader_workers()))
    

### set the training parameters

In [67]:
batch_size = [128]
epochs = [5]
rounds = 1
lr = 0.01

## Train Function

#### using pynvml to get the GPU power consumption

In [68]:
def nvml_sampling_thread(handle, filename, stop_event, sampling_interval):
    """
    在单独的线程中定期调用 NVML, 获取功耗数据并存储到 data_queue 中。
    参数：
    - handle: nvmlDeviceGetHandleByIndex(0) 得到的 GPU 句柄
    - data_queue: 用于存放 (timestamp, power_in_watts) 数据的队列
    - stop_event: 当此事件被设置时，线程应结束循环
    - sampling_interval: 采样间隔（秒）
    """
    with open(filename/'energy_consumption_file.csv', 'a') as f:  # 追加模式
        # 写入列名
        f.write("timestamp,power_in_watts\n")
        while not stop_event.is_set():
            try:
                # 采集功率和时间戳
                current_time = time.time()
                current_power = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000.0  # 转换 mW -> W
                # 写入文件
                f.write(f"{current_time},{current_power}\n")
                # 等待下一次采样
                time.sleep(sampling_interval)
            except pynvml.NVMLError as e:
                print(f"NVML Error: {e}")
                break

#### set the training process

In [69]:
def train_func(net, train_iter, test_iter, num_epochs, lr, device, filename, sampling_interval):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    # print(f'The name of the layers are: {alexlayer}')
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    # save all epochs time data using list
    to_device_intervals_total = []
    forward_intervals_total = []
    loss_intervals_total = []
    backward_intervals_total = []
    optimize_intervals_total = []
    test_intervals_total = []

    # create a dictionary to store each layer time period data in each batch
    layer_time = {}

    # create a list to store the epoch time data
    epoch_intervals_total = []
    
    # 初始化NVML和采样线程
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    power_data_queue = queue.Queue()
    stop_event = threading.Event()
    sampler_thread = threading.Thread(target=nvml_sampling_thread, args=(handle, filename, stop_event, sampling_interval))
    sampler_thread.start()

    for epoch in range(num_epochs):
        layer_time[str(epoch)] = {}

        print('The epoch is:', epoch+1)
        metric = d2l.Accumulator(3)  # train_loss, train_acc, num_examples
        to_device_intervals_epoch = []  # 用来记录本epoch每个batch的to_device时间段
        forward_intervals_epoch = []  # 用来记录本epoch每个batch的forward时间段
        loss_intervals_epoch = []  # 用来记录本epoch每个batch的loss时间段
        backward_intervals_epoch = [] 
        optimize_intervals_epoch = []
        test_intervals_epoch = []   
        epoch_intervals_epoch = []  # 用来记录本epoch的时间段

        epoch_start_time = time.time()

        net.train()
        for i, (X, y) in enumerate(train_iter):
            print('The batch is:', i+1)

            layer_time[str(epoch)][str(i)] = {}
            optimizer.zero_grad()
            torch.cuda.synchronize()

            # 记录to_device前后的时间戳
            start_ttd_time = time.time()
            X, y = X.to(device), y.to(device)
            torch.cuda.synchronize()
            end_ttd_time = time.time()
            to_device_intervals_epoch.append((start_ttd_time, end_ttd_time))

            # forward
            start_forward_time = time.time()
            y_hat = X

            '''alexnet'''
            # layer_count = 0

            '''resnet18'''
            block_count = 0

            for block in net:
                name = block.__class__.__name__ # get the name of the layer
                name_show = name + '_' + str(block_count)
                print(f'The name of the layer is: {name_show}')
                layer_time[str(epoch)][str(i)][name_show] = {}
                '''alexnet'''
                # name = name + '_' + str(layer_count)
                # layer_count += 1
                # if name in layerlist_alexnet:
                #     start_layer_time = time.time()
                #     y_hat = layer(y_hat)
                #     torch.cuda.synchronize()
                #     end_layer_time = time.time()
                #     layer_time[str(epoch)][str(i)][name] = (start_layer_time, end_layer_time)
                '''resnet18'''
                # if name == 'Sequential':
                #     resblock_num = 0
                #     # iterate the subblock in the block
                #     for subblock in block:
                #         subblock_name = subblock.__class__.__name__
                #         # determine if the layer is Residual18
                #         if subblock_name == 'Residual18':
                #             # clear the layer_time
                #             subblock.layer_time = {}
                #             y_hat = subblock(y_hat)
                #             torch.cuda.synchronize()
                #             print(subblock.layer_time)
                #             subblock_name = subblock_name + '_' + str(resblock_num)
                #             layer_time[str(epoch)][str(i)][name_show][subblock_name] = subblock.layer_time
                #         # if the inner layer is not Residual18
                #         else:
                #             # print the current layer name
                #             print(subblock_name)
                #             start_layer_time = time.time()
                #             y_hat = subblock(y_hat)
                #             torch.cuda.synchronize()
                #             end_layer_time = time.time()
                #             layer_time[str(epoch)][str(i)][name_show][subblock_name] = (start_layer_time, end_layer_time)
                #         resblock_num += 1
                #     block_count += 1
                # else:
                #     start_layer_time = time.time()
                #     y_hat = block(y_hat)
                #     torch.cuda.synchronize()
                #     end_layer_time = time.time()
                #     layer_time[str(epoch)][str(i)][name_show] = (start_layer_time, end_layer_time)
                '''googlenet'''
                if name == 'Sequential':
                    IncepNum = 0
                    for subblock in block:
                        subblock_name = subblock.__class__.__name__
                        if subblock_name == 'Inception':
                            subblock.layer_time = {}
                            y_hat = subblock(y_hat)
                            torch.cuda.synchronize()
                            subblock_name = subblock_name + '_' + str(IncepNum)
                            layer_time[str(epoch)][str(i)][name_show][subblock_name] = subblock.layer_time
                            IncepNum += 1
                        else:
                            start_layer_time = time.time()
                            y_hat = subblock(y_hat)
                            torch.cuda.synchronize()
                            end_layer_time = time.time()
                            layer_time[str(epoch)][str(i)][name_show][subblock_name] = (start_layer_time, end_layer_time)
                    block_count += 1
                else:
                    start_layer_time = time.time()
                    y_hat = block(y_hat)
                    torch.cuda.synchronize()
                    end_layer_time = time.time()
                    layer_time[str(epoch)][str(i)][name_show] = (start_layer_time, end_layer_time)

            # y_hat = net(X)
            # torch.cuda.synchronize()
            end_forward_time = time.time()
            forward_intervals_epoch.append((start_forward_time, end_forward_time))

            # loss
            start_loss_time = time.time()
            l = loss_fn(y_hat, y)
            torch.cuda.synchronize()
            end_loss_time = time.time()
            loss_intervals_epoch.append((start_loss_time, end_loss_time))

            # backward
            start_backward_time = time.time()
            l.backward()
            torch.cuda.synchronize()
            end_backward_time = time.time()
            backward_intervals_epoch.append((start_backward_time, end_backward_time))

            # optimize
            start_optimize_time = time.time()
            optimizer.step()
            torch.cuda.synchronize()
            end_optimize_time = time.time()
            optimize_intervals_epoch.append((start_optimize_time, end_optimize_time))

            with torch.no_grad():
                metric.add(l*X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            train_acc = metric[1] / metric[2]

        start_test_time = time.time()
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        end_test_time = time.time()
        print(f'train acc {train_acc:.3f}, test acc {test_acc:.3f}')
        test_intervals_epoch.append((start_test_time, end_test_time))

        epoch_end_time = time.time()
        epoch_intervals_epoch.append((epoch_start_time, epoch_end_time))

        # data need to be saved
        # add the intervals_epoch to intervals_total
        to_device_intervals_total.append(to_device_intervals_epoch)
        forward_intervals_total.append(forward_intervals_epoch)
        loss_intervals_total.append(loss_intervals_epoch)
        backward_intervals_total.append(backward_intervals_epoch)
        optimize_intervals_total.append(optimize_intervals_epoch)
        test_intervals_total.append(test_intervals_epoch)
        epoch_intervals_total.append(epoch_intervals_epoch)


    # 训练结束后关闭线程
    stop_event.set()
    sampler_thread.join()

    pynvml.nvmlShutdown()

    return to_device_intervals_total, forward_intervals_total, loss_intervals_total, backward_intervals_total, optimize_intervals_total, test_intervals_total, epoch_intervals_total, layer_time

## Start to train the model

### set a function to train the model with FashionMNIST datasets

In [70]:
def train_model(main_folder, batch_size, num_epochs, round, lr, device, sample_interval, net):
    print(f'The epoch is set: {num_epochs}, batch is set: {batch_size}, is in {round+1}th running')
    # create the folder to store the data
    # epoch_batch_folder = main_folder/f'E{num_epochs}_B{batch_size}_R{round}'
    sr_number = int(sample_interval*1000)
    epoch_batch_folder = f'E{num_epochs}_B{batch_size}_R{round}_SR{sr_number}_layer'

    data_dir = 'fashion_mnist'
    # data_dir = 'cifar100'
    # data_dir = 'cifar10'

    # the folder path is main_folder/epoch_batch_folder
    folder_path = main_folder/epoch_batch_folder/data_dir
    print(f'The folder path is: {folder_path}')
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    else:
        pass
    
    train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)
    # show the shape of the data
    list_of_i = []
    for i, (X, y) in enumerate(train_iter):
        if i < 3:
            print('the shape of the', i, 'batch of the train_iter is:', X.shape)
        else:
            pass
        list_of_i.append(i)
    print(f'The number of batches is: {np.array(list_of_i).shape}')
    to_device_intervals_total, forward_intervals_total, \
        loss_intervals_total, backward_intervals_total,\
            optimize_intervals_total, test_intervals_total, \
                epoch_intervals_total, layer_time_data = train_func(net, train_iter, test_iter, \
                                                                    num_epochs, lr, device, folder_path, sample_interval)
    
    to_device_data = np.array(to_device_intervals_total)
    forward_time = np.array(forward_intervals_total)
    loss_time = np.array(loss_intervals_total)
    backward_time = np.array(backward_intervals_total)
    optimize_time = np.array(optimize_intervals_total)
    test_time = np.array(test_intervals_total)
    epoch_time = np.array(epoch_intervals_total)

    # save the layer_time_alexnet, the type is a dictionary, need to be saved as a csv file
    # the first column is the epoch, the second column is the batch, the third column is the layer name, the fourth column is the start time, the fifth column is the end time
    layer_time_data_df = pd.DataFrame.from_dict(layer_time_data)
    


    # print(layer_time_alexnet_df)

    # save the data
    np.save(folder_path/'to_device.npy', to_device_data, allow_pickle=True)
    np.save(folder_path/'forward.npy', forward_time, allow_pickle=True)
    np.save(folder_path/'loss.npy', loss_time, allow_pickle=True)
    np.save(folder_path/'backward.npy', backward_time, allow_pickle=True)
    np.save(folder_path/'optimize.npy', optimize_time, allow_pickle=True)
    np.save(folder_path/'test.npy', test_time, allow_pickle=True)
    np.save(folder_path/'epoch.npy', epoch_time, allow_pickle=True)

    # save the data to csv file
    layer_time_data_df.to_csv(folder_path/'layer_time.csv', index=False)


### run the model

In [71]:
# sampling_interval = 0.002 # 2ms
sampling_interval = 0.002 # 1ms
# create the folder to store the data
main_folder = DataList[0]

print('The folder is:', main_folder)
    # find out that if the folder exists in the data path
    # 判断文件是否存在
if main_folder.exists():
    print("文件存在。")
else:
    os.makedirs(main_folder)
    print("文件不存在，已创建。")
    print("文件创建于：", main_folder)
for epoch in epochs:
    for batch in batch_size:
        for round in range(rounds):
            train_model(main_folder, batch, epoch, round, lr, device, sampling_interval, googlenet_f)

The folder is: /root/autodl-tmp/GreenAI/3080/ModelsData/googlenet_origin
文件存在。
The epoch is set: 5, batch is set: 128, is in 1th running
The folder path is: /root/autodl-tmp/GreenAI/3080/ModelsData/googlenet_origin/E5_B128_R0_SR2_layer/fashion_mnist


the shape of the 0 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 1 batch of the train_iter is: torch.Size([128, 1, 224, 224])
the shape of the 2 batch of the train_iter is: torch.Size([128, 1, 224, 224])
The number of batches is: (469,)
training on cuda
The epoch is: 1
The batch is: 1
The name of the layer is: Sequential_0
The name of the layer is: Sequential_1
The name of the layer is: Sequential_2
The name of the layer is: Sequential_3
The name of the layer is: Sequential_4
The name of the layer is: Linear_5
The batch is: 2
The name of the layer is: Sequential_0
The name of the layer is: Sequential_1
The name of the layer is: Sequential_2
The name of the layer is: Sequential_3
The name of the layer is: Sequential_4
The name of the layer is: Linear_5
The batch is: 3
The name of the layer is: Sequential_0
The name of the layer is: Sequential_1
The name of the layer is: Sequential_2
The name of the layer is: Sequential_3
The name of the layer is: Sequential